To-do:
1. Текст был очищен только от одного мусорного элемента в качестве примера. Исслудйте данные через ноутбук или чере веб-интерфейс BigQuery на предмет других мусорных элементов в тексте, которые не несут в себе никакого особого смысла, а только создают шум в данных. Доработайте функцию очистки тектосвых данных, чтобы в нее можно было передать список ненужного мусора и разом выполнялась очистка
2. Проведите стратифицировнную кросс-валидуцию нейросетевого классификатора https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.StratifiedKFold.html
3. Поэксперементируйте с гиперпараметрами нейросетевого классификатора, постарайтесь повысить качество его работы
--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
4. Попробуйте использовать не Word2Vec для получения векторого представления текста, а TF-IDF преобразование http://zabaykin.ru/?p=558 http://nlpx.net/archives/57
5. Попробуйте использовать более тонко настриваемые алгоритмы нейросетей, например из этого видео https://www.youtube.com/watch?v=cPkH1k3U1c8 

In [1]:
!pip install langdetect

In [2]:
import warnings
warnings.filterwarnings('ignore')

from google.oauth2 import service_account
import pandas_gbq 

import numpy as np
import pandas as pd
import math as mt
import datetime as dt
import nltk
from nltk.corpus import stopwords
from langdetect import detect
import re
import string
from string import digits

from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report,confusion_matrix
from gensim.models.word2vec import Word2Vec
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import roc_curve,auc
import matplotlib.pyplot as plt
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from collections import Counter
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
CREDENTIALS = service_account.Credentials.from_service_account_info({
})

In [4]:
STOPWORDS = set(stopwords.words('english'))

In [5]:
#funtion for getting fresh data from DWH for workload model
"""[summary]
Funtion for getting fresh data from BigQuery for workload scoring model
[description]
Credentials - google service account object with credentials data for project
[example]
Input: Credentials = credentials_object
Output: description	                                        channel	 category	category_flag
        \nChat transcript:\nVisitor: I want to buy wit...	chat	 ps	        1
        \nChat transcript:\nVisitor: hell i had a prob...	chat	 ps	        1
        \nChat transcript:\nVisitor: لا استطيع الشراء ...	     chat	  ps	     1
"""
def getDwhData(Credentials):
    statement_bigquery_sql = " ".join(["select description, channel, case",
                                       "when manual_category in ('payment_problem','how_to_pay','howtopay','how_to_play','paystation_error','ps_problem','ps_declined') then 'ps'",
                                       "else 'other'",
                                       "end as category,",
                                       "case",
                                       "when manual_category in ('payment_problem','how_to_pay','howtopay','how_to_play','paystation_error','ps_problem','ps_declined') then 0",
                                       "else 1",
                                       "end as category_flag",
                                       "from `xsolla_summer_school.customer_support`",
                                       "where manual_category is not null and",
                                       "manual_category <> '' and",
                                       "description is not null and",
                                       "description <> '' and",
                                       "channel is not null and",
                                       "channel <> '' and",
                                       "channel in ('chat','facebook')"])
    
    dataframe_bigquery = pandas_gbq.read_gbq(statement_bigquery_sql,project_id='findcsystem', credentials=Credentials, dialect='standard')

    return dataframe_bigquery


"""[summary]
Function for transform text to lower case
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["Text_1","Text_2"]
Output: ["text_1","text_2"]
"""
def lowerCase(Corpus):
    corpus = [i.lower().replace('\n',' ') for i in Corpus]
    return corpus





PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(Corpus):
    """custom function to remove the punctuation"""
    corpus = [i.translate(str.maketrans('', '', PUNCT_TO_REMOVE)) for i in Corpus]
    return corpus

STOPWORDS = set(stopwords.words('english'))
STOPWORDS.add('hi')
STOPWORDS.add("i'm")
def remove_stopwords(Corpus):
    """custom function to remove the stopwords"""
    return [" ".join([word for word in str(i).split() if word not in STOPWORDS]) for i in Corpus]


def remove_freqwords(Corpus):
    cnt = Counter()
    for text in Corpus:
        for word in text.split():
            cnt[word] += 1
    FREQWORDS = set([w for (w, wc) in cnt.most_common(30)])
    return [" ".join([word for word in str(i).split() if word not in FREQWORDS]) for i in Corpus]

def remove_urls(Corpus):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    html_pattern = re.compile('<.*?>')
    return_text=[]
    for text in Corpus:
        text=url_pattern.sub(r'', text)
        text=html_pattern.sub(r'',text)
        return_text.append(text)
    return return_text
    

def remove_numbers(Corpus):
    return [''.join(i for i in text if not i.isdigit()) for text in Corpus]


def Stem(Corpus):
    stemmer = PorterStemmer()
    return [' '.join([stemmer.stem(word) for word in text.split()]) for text in Corpus]



"""[summary]
Function for getting language of text
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["Text_1","Text_2"]
Output: ["en","ru"]
"""
def getTextLanguage(Corpus):
    txt_lang = []
    for txt in Corpus:
        try:
            lang = detect(txt)
            txt_lang.append(lang)
        except:
            lang = 'error'
            txt_lang.append(lang)
    
    return txt_lang


"""[summary]
Function for tokenization text
[description]
Corpus - list or array object, with text data
[example]
Input: Corpus = ["word1 word2","word3 word4"]
Output: [["word1","word2"],["word3","word4"]]
"""  
def textToTokens(Corpus):
    corpus = [i.split() for i in Corpus]
    return corpus 


"""[summary]
Function for clear text after garbage
[description]
Corpus - list or array object, with text data
Substr - string, regular expression
[example]
Input: Corpus = [["word1","word2"],["word3","word4"]]
       Substr = r'word1
Output: [["word2"],["word3","word4"]]
"""  
def clearTextAfterGarbage(Corpus):
    # clear_corpus = []
    # for text in Corpus:
    #     indexes = []
    #     text_len = len(text)
    #     try:
    #         for i in range(0,text_len):
    #             res = re.search(Substr,text[i])
    #             if res != None:
    #                 indexes.append(i)
                
    #         #delete garbage word from text
    #         for index in indexes:
    #             del text[index]
        
    #         clear_corpus.append(text)
    #     except:
    #         clear_corpus.append("error")
    corpus = lowerCase(Corpus)
    corpus = remove_urls(corpus)
    corpus = remove_numbers(corpus)
    corpus = remove_stopwords(corpus)
    corpus = remove_punctuation(corpus)
    corpus = Stem(corpus)
    corpus = remove_freqwords(corpus)
    return corpus


"""[summary]
Build word vector by using pre-trained Word2Vec model
[description]
Size - lenght of vector
Word2Vec_Model - gensim object
"""  
def buildWordVector(Text,Size,Word2Vec_Model):
    vec = np.zeros(Size).reshape((1,Size))
    count = 0.

    for word in Text:
        try:
            vec += Word2Vec_Model[word].reshape((1,Size))
            count += 1.
        except KeyError:
            continue
    
    if count != 0:
        vec /= count
    
    return vec

# RAWDATA 

In [6]:
#getting data from dwh
SupportRawDataframe = getDwhData(CREDENTIALS)
SupportRawDataframe.shape

(23450, 4)

In [7]:
SupportRawDataframe.head(10)

,description,channel,category,category_flag
0,\nChat transcript:\nVisitor: I want to buy wit...,chat,ps,0
1,\nChat transcript:\nVisitor: hell i had a prob...,chat,ps,0
2,\nChat transcript:\nVisitor: لا استطيع الشراء ...,chat,ps,0
3,\nChat transcript:\nVisitor: im having trouble...,chat,ps,0
4,\nChat transcript:\nVisitor: Hi\nAna: Hello. H...,chat,ps,0
5,\nChat transcript:\nVisitor: Здраствуйте\nAna:...,chat,ps,0
6,\nChat transcript:\nVisitor: hello\nVisitor: w...,chat,ps,0
7,\nChat transcript:\nVisitor: not letting me bu...,chat,ps,0
8,\nChat transcript:\nVisitor: مرحبا\nAna: Hello...,chat,ps,0
9,\nChat transcript:\nVisitor: hi im trying to b...,chat,ps,0


In [8]:
SupportRawDataframe.tail(10)

,description,channel,category,category_flag
23440,"\nChat transcript:\nVisitor: Hello, I bought p...",chat,other,1
23441,\nChat transcript:\nVisitor: Good day\n\nI acc...,facebook,other,1
23442,\nChat transcript:\nVisitor: payment not going...,chat,other,1
23443,\nChat transcript:\nVisitor: Paid for faceit s...,chat,other,1
23444,"\nChat transcript:\nVisitor: Hello, I am tryin...",chat,other,1
23445,"\nChat transcript:\nVisitor: Hi, i made a pruc...",chat,other,1
23446,"\nChat transcript:\nVisitor: Hi, how long will...",chat,other,1
23447,\nChat transcript:\nVisitor: I bought playerun...,chat,other,1
23448,\nChat transcript:\nVisitor: Good day i took t...,chat,other,1
23449,\nChat transcript:\nVisitor: hi\nVisitor: hell...,chat,other,1


# DATA PREPROC

In [9]:
data = getDwhData(CREDENTIALS)

In [10]:
#transform text to lower case
corpus = data.description
corpus.astype('str')

corpus = lowerCase(corpus)
corpus = remove_urls(corpus)
corpus = remove_numbers(corpus)
corpus = remove_stopwords(corpus)
corpus = remove_punctuation(corpus)
corpus = Stem(corpus)
corpus = remove_freqwords(corpus)
#getting language for text corpus
corpus_lang = getTextLanguage(corpus)
corpus

['want buy paypal show problem call support would error code text screenshot error possibl file request reload wait file request file ok send problem stand onlin take time resolv patienc ill updat minut solv stand onlin take time resolv patienc ill updat minut reload solv clear cach cooki browser histori log out log back allow refresh solv tri log anoth browser opera googl chrome firefox reload ok reload file attach messag',
 'hell problem yesterday subscrib peopl got tri subscrib time got nickt gift sub channel say subscrib anymor mean twitch subscript ye could link twitch uk spoke yesterday stand onlin take time resolv patienc ill updat minut okay accord system credit balanc twitch redeem balanc partli fulli next twitch order choos subscript tier period option click pay see deduct price subscript understand okay anyth els with nope that nice',
 'لا استطيع الشراء ومعلومات الشراء صحيحة ميه بالمية مرحباً ، للأسف ، لا أتحدث العربية وسوف استخدم المترجم للتواصل أو يمكننا الاستمرار باللغة ا

In [11]:
#new dataframe with texts in lower case, without /n symbol and with lang for text
SupportRawDataframe['description'] = corpus
SupportRawDataframe['lang'] = corpus_lang

In [12]:
SupportRawDataframe.head(10)

,description,channel,category,category_flag,lang
0,want buy paypal show problem call support woul...,chat,ps,0,en
1,hell problem yesterday subscrib peopl got tri ...,chat,ps,0,en
2,لا استطيع الشراء ومعلومات الشراء صحيحة ميه بال...,chat,ps,0,en
3,im troubl xsolla auth digit code make pymaent ...,chat,ps,0,en
4,cant pay paypal becous ad credit card pay payp...,chat,ps,0,en
5,здраствуйте здравствуйте чем могу вам помочь я...,chat,ps,0,ru
6,cant pay whit mobil phone im frome lithuania b...,chat,ps,0,en
7,let buy anyth wait moment check unfortun proce...,chat,ps,0,en
8,مرحبا problem tri credit card look like exceed...,chat,ps,0,en
9,im tri buy v buck wont let pay use amazon gift...,chat,ps,0,en


In [13]:
SupportRawDataframe.tail(10)

,description,channel,category,category_flag,lang
23440,bought pubg servic run well comput wonder coul...,chat,other,1,en
23441,good accident v subscript faceit instead souze...,facebook,other,1,en
23442,go eugen eugen one moment sir ill check okay e...,chat,other,1,en
23443,paid faceit subscript activ yet georg georg di...,chat,other,1,en
23444,tri upgrad copi escap tarkov keep revers haile...,chat,other,1,en
23445,hi made pruchas dollar send bitcoin cash walle...,chat,other,1,en
23446,hi long take friend god pack michael it contai...,chat,other,1,en
23447,bought playerunknown battleground would like r...,chat,other,1,en
23448,good took wrong packag want take premuim took ...,chat,other,1,en
23449,one answer,chat,other,1,en


In [14]:
#getting only en texts
SupportDataframe_eng = SupportRawDataframe[SupportRawDataframe.lang == 'en'][:]
SupportDataframe_eng

,description,channel,category,category_flag,lang
0,want buy paypal show problem call support woul...,chat,ps,0,en
1,hell problem yesterday subscrib peopl got tri ...,chat,ps,0,en
2,لا استطيع الشراء ومعلومات الشراء صحيحة ميه بال...,chat,ps,0,en
3,im troubl xsolla auth digit code make pymaent ...,chat,ps,0,en
4,cant pay paypal becous ad credit card pay payp...,chat,ps,0,en
...,...,...,...,...,...
23445,hi made pruchas dollar send bitcoin cash walle...,chat,other,1,en
23446,hi long take friend god pack michael it contai...,chat,other,1,en
23447,bought playerunknown battleground would like r...,chat,other,1,en
23448,good took wrong packag want take premuim took ...,chat,other,1,en


In [15]:
#text tekenization
tokenization = textToTokens(SupportDataframe_eng.description)
SupportDataframe_eng['description'] = tokenization

In [16]:
SupportDataframe_eng['description'].shape

(14093,)

In [17]:
text= SupportDataframe_eng['description']
text.shape

(14093,)

# TRAIN TEST SPLIT DATAFRAMES

In [18]:
#list of unique categories
unique_categories = np.unique(SupportDataframe_eng.category)
unique_categories

array(['other', 'ps'], dtype=object)

In [19]:
descriptions = SupportDataframe_eng['description']
categories = SupportDataframe_eng['category_flag']
XTrain,XTest,YTrain,YTest = train_test_split(descriptions,
                                             categories,
                                             stratify = categories,
                                             test_size = 0.2,
                                             random_state = 40)

# FROM TEXTS TO VECTORS

In [20]:
#initialize Word2Vec model for embedding words to vectors
NDim = 100
Imdb_w2v = Word2Vec(size = NDim,min_count = 10)
Imdb_w2v.build_vocab(descriptions)

In [21]:
Imdb_w2v.train(descriptions,total_examples = Imdb_w2v.corpus_count,epochs = Imdb_w2v.epochs)

(4070833, 5040500)

In [22]:
#embedding training messages to vectors for neutral classifier
TrainVecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in XTrain])

In [23]:
TrainVecs

array([[ 0.63196336,  0.42804588,  0.3344485 , ...,  0.01479577,
         0.54894453,  0.67878505],
       [ 0.63818794,  0.40601178, -0.37199388, ...,  0.02886905,
         0.50088494,  0.28708298],
       [ 0.14747235,  0.08723446, -0.17388005, ...,  0.18353763,
         0.14121595,  0.17064925],
       ...,
       [ 0.42384902,  0.40516275, -0.85429941, ...,  0.95599675,
         1.09479232,  0.15219155],
       [ 0.91614844,  0.43148805,  0.21159645, ...,  0.24196654,
        -0.12318984, -0.02645644],
       [ 0.68348039, -0.07393169, -0.42695212, ...,  0.0755152 ,
        -0.23747323,  0.07978688]])

In [24]:
TestVecs = np.concatenate([buildWordVector(i,NDim,Imdb_w2v) for i in XTest])

# CROSSVALIDATION AND BUILD CLASSIFIER

In [25]:
TextClassifier = MLPClassifier(hidden_layer_sizes = (20,10), max_iter = 10000, random_state = 40, n_iter_no_change=100, early_stopping=True)

In [26]:
skf = StratifiedKFold(n_splits=5)
Scores = cross_val_score(TextClassifier, TrainVecs, YTrain, cv = skf)
print(Scores)
print(np.mean(Scores))

[0.91973392 0.92062084 0.91263858 0.91662971 0.92635315]
0.9191952419645194


In [27]:
TextClassifier.fit(TrainVecs,YTrain)

MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
              beta_2=0.999, early_stopping=True, epsilon=1e-08,
              hidden_layer_sizes=(20, 10), learning_rate='constant',
              learning_rate_init=0.001, max_fun=15000, max_iter=10000,
              momentum=0.9, n_iter_no_change=100, nesterovs_momentum=True,
              power_t=0.5, random_state=40, shuffle=True, solver='adam',
              tol=0.0001, validation_fraction=0.1, verbose=False,
              warm_start=False)

In [28]:
pred = TextClassifier.predict(TestVecs)
print(confusion_matrix(YTest,pred))
print(classification_report(YTest,pred))

[[ 415  132]
 [ 103 2169]]
              precision    recall  f1-score   support

           0       0.80      0.76      0.78       547
           1       0.94      0.95      0.95      2272

    accuracy                           0.92      2819
   macro avg       0.87      0.86      0.86      2819
weighted avg       0.92      0.92      0.92      2819



In [29]:
# .95 .78

# reverse engineering

In [30]:
YTest.index

Int64Index([13378,  5170,  4205, 15009,   926, 20897, 22337, 15847,  7712,
            21324,
            ...
             1464,  5051,  7811, 21714,  7249, 13502, 19412, 11786, 20694,
            17645],
           dtype='int64', length=2819)

In [31]:
pred.shape

(2819,)

In [32]:
X=pd.DataFrame(XTest)

In [33]:
X['y']=YTest
X['pred']=pred

In [34]:
for i in X[X['y']!=X['pred']]['description']:
    print(i)

['tri', 'buy', 'one', 'year', 'vip', 'want', 'dni', 'reload']
['tri', 'subscript', 'twitch', 'work', 'long', 'suddenli', 'one', 'anoth', 'person', 'sub', 'person', 'got', 'happi', 'assist', 'let', 'check', 'cannot', 'subscrib', 'anyon', 'unfortun', 'secur', 'system', 'detect', 'latest', 'atyp', 'chang', 'secur', 'set', 'retri', 'also', 'see', 'use', 'card', 'cancel', 'bank', 'retri', 'use', 'option', 'u', 'let', 'tri', 'sure', 'take', 'time', 'let', 'know', 'difficulti', 'least', 'process', 'now', 'gotta', 'wait', 'blush', 'atyp', 'accid', 'ok', 'work', 'fast', 'repli', 'actual', 'usual', 'happen', 'make', 'sever', 'attempt', 'short', 'amount', 'time', 'also', 'see', 'use', 'differ', 'method', 'could', 'also', 'trigger', 'secur', 'system', 'worri', 'next', 'hour', 'go', 'normal', 'feel', 'free', 'anytim', 'case', 'concern', 'happi', 'also', 'glad', 'hear', 'let', 'know', 'anyth', 'els', 'could', 'assist', 'ye', 'like', 'sub', 'train', 'twitch', 'channel', 'tab', 'open', 'readi', 'sub',

In [35]:
cnt = Counter()
for text in X[X['y']!=X['pred']]['description']:
    for word in text:
        cnt[word] += 1

In [36]:
cnt.most_common(50)

[('card', 294),
 ('tri', 228),
 ('see', 214),
 ('reload', 199),
 ('bank', 176),
 ('use', 160),
 ('wait', 156),
 ('check', 153),
 ('paypal', 137),
 ('system', 131),
 ('process', 131),
 ('ok', 127),
 ('time', 124),
 ('could', 116),
 ('hailey', 113),
 ('charg', 112),
 ('one', 108),
 ('happi', 108),
 ('moment', 108),
 ('pay', 106),
 ('code', 105),
 ('unfortun', 99),
 ('take', 99),
 ('ye', 99),
 ('secur', 96),
 ('buy', 95),
 ('make', 94),
 ('amount', 94),
 ('would', 92),
 ('need', 91),
 ('alexandra', 91),
 ('error', 90),
 ('inform', 88),
 ('get', 87),
 ('minut', 87),
 ('it', 84),
 ('michael', 83),
 ('say', 82),
 ('option', 81),
 ('sorri', 78),
 ('xsolla', 77),
 ('support', 76),
 ('let', 75),
 ('money', 75),
 ('file', 73),
 ('know', 72),
 ('complet', 72),
 ('want', 71),
 ('problem', 71),
 ('back', 71)]